In [3]:
clear all

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io 
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from scipy import signal
from scipy.fft import fft
import random
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [10]:
faultI10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_10hz.mat')
faultI30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_30hz.mat')
faultII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_10hz.mat')
faultII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_30hz.mat')
faultIII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_10hz.mat')
faultIII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_30hz.mat')
faultIV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_10hz.mat')
faultIV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_30hz.mat')
faultV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_10hz.mat')
faultV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_30hz.mat')
# faultVI10=scipy.io.loadmat('C:/Users/PikPik/Desktop/Rotating Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_6_10hz.mat')
# faultVI30=scipy.io.loadmat('C:/Users/PikPik/Desktop/Rotating Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_6_30hz.mat')

In [11]:
sigII30=faultII30['Acquisition_un_2_30hz']
sigII10=faultII10['Acquisition_un_2_10hz']
del faultII30,faultII10
sigI30=faultI30['Acquisition_un_1_30hz']
sigI10=faultI10['Acquisition_un_1_10hz']
del faultI30,faultI10
sigIII10=faultIII10['Acquisition_un_3_10hz']
sigIII30=faultIII30['Acquisition_un_3_30hz']
del faultIII10,faultIII30
sigIV10=faultIV10['Acquisition_un_4_10hz']
sigIV30=faultIV30['Acquisition_un_4_30hz']
del faultIV10,faultIV30
sigV10=faultV10['Acquisition_un_5_10hz']
sigV30=faultV30['Acquisition_un_5_30hz']
del faultV10,faultV30
# sigVI10=faultVI10['Acquisition_un_6_10hz']
# sigVI30=faultVI30['Acquisition_un_6_30hz']
# del faultVI10,faultVI30

In [14]:

def sliding_window(data):
    """
    Applies a sliding window to the given data with the specified window size
    and stride length. Padding is added at the beginning and end of the data to
    ensure all samples are included in windows.
    """
    # num_padding = window_size - stride
    # data_padded = np.pad(data, ((num_padding, num_padding), (0, 0)), 'constant')
    # window_data = []
    # for i in range(0, len(data_padded) - window_size + 1, stride):
    #     window = data_padded[i:i+window_size, :]
    #     window_data.append(window)
    # window_data = np.array(window_data)
    # window_data = window_data[:training_size, :, :]
    # window_data = np.transpose(window_data, (0, 2, 1))
    # return window_data
    # Assuming that you have your data stored in a numpy array called "data"
    fs = 1000 # Sampling frequency of the data
    window_size = 256 # Number of samples in each window
    overlap = 128 # Overlap between windows

    # Define the window function to be used for the STFT
    window = signal.hamming(window_size)

    # Compute the STFT using the hamming window
    f, t, Zxx = signal.stft(data, fs=fs, window=window, nperseg=window_size, noverlap=overlap)

    
    num_padding = window_size - overlap
    data_padded = np.pad(data, ((num_padding, num_padding), (0, 0)), 'constant')
    window_data = []
    for i in range(0, len(data_padded) - window_size + 1, stride):
        window = data_padded[i:i+window_size, :]
        window_data.append(window)
    window_data = np.array(window_data)
    
    window_data = np.transpose(window_data, (0, 2, 1))
    
    
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=np.max(np.abs(Zxx)), shading='gouraud')
    plt.xlabel('Time [sec]')
    plt.ylabel('Frequency [Hz]')
    plt.colorbar()
    plt.show()
    
    return window_data
    # Plot the result

In [15]:
sigtI10hz=sliding_window(sigI10)[:,1:13,:]
sigtI30hz=sliding_window(sigI30)[:,1:13,:]
sigtII10hz=sliding_window(sigII10)[:,1:13,:]
sigtII30hz=sliding_window(sigII30)[:,1:13,:]
del sigI10,sigI30,sigII10,sigII30
sigtIII30hz=sliding_window(sigIII30)[:,1:13,:]
sigtIII10hz=sliding_window(sigIII10)[:,1:13,:]
sigtIV30hz=sliding_window(sigIV30)[:,1:13,:]
sigtIV10hz=sliding_window(sigIV10)[:,1:13,:]
del sigIII30,sigIII10,sigIV30,sigIV10
sigtV30hz=sliding_window(sigV30)[:,1:13,:]
sigtV10hz=sliding_window(sigV10)[:,1:13,:]
# sigtVI30hz=sliding_window(sigVI30)
# sigtVI10hz=sliding_window(sigVI10)
# del sigV30,sigV10,sigVI30,sigVI10

ValueError: window is longer than input signal

In [7]:
from scipy import signal
from scipy.signal import butter, filtfilt

In [8]:
np.shape(sigtI10hz)

(1506, 12, 500)

In [9]:
def normalize_last_dim(data, epsilon=0.01):
    # compute mean and standard deviation along the last dimension of the data
    last_dim_mean = np.mean(data, axis=-1, keepdims=True)
    last_dim_std = np.std(data, axis=-1, keepdims=True)

    # normalize the last dimension using the mean and standard deviation
    normalized_data = (data- last_dim_mean) /last_dim_std+epsilon

#     # stack the normalized values with the original values along the last dimension
    # normalized_data = np.concatenate([data[..., :-1], normalized_data[..., np.newaxis]], axis=-1)

    return normalized_data

In [10]:
a1n=normalize_last_dim(sigtI10hz)
del sigtI10hz
a2n=normalize_last_dim(sigtI30hz)
del sigtI30hz
a3n=normalize_last_dim(sigtII10hz)
del sigtII10hz
a4n=normalize_last_dim(sigtII30hz)
del sigtII30hz
a5n=normalize_last_dim(sigtIII10hz)
del sigtIII10hz
a6n=normalize_last_dim(sigtIII30hz)
del sigtIII30hz
a7n=normalize_last_dim(sigtIV10hz)
del sigtIV10hz
a8n=normalize_last_dim(sigtIV30hz)
del sigtIV30hz
a9n=normalize_last_dim(sigtV10hz)
del sigtV10hz
a10n=normalize_last_dim(sigtV30hz)
del sigtV30hz

In [11]:
datalable1=np.concatenate([a1n,a2n],axis=-1)
del a1n,a2n
datalable2=np.concatenate([a3n,a4n],axis=-1)
del a3n,a4n
datalable3=np.concatenate([a5n,a6n],axis=-1)
del a5n,a6n
datalable4=np.concatenate([a7n,a8n],axis=-1)
del a7n,a8n
datalabele5=np.concatenate([a9n,a10n],axis=-1)
del a9n,a10n

In [12]:
labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)
dataall=np.concatenate([datalable1,datalable2,datalable3,datalable4,datalabele5],axis=0)

In [13]:
indices = np.random.permutation(dataall.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = dataall[indices]
shuffled_labels = labels[indices]

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split

def split_data_equally(X, y, test_size=0.4, random_state=None):
    # Get unique labels in y and their indices
    labels, indices = np.unique(y, return_index=True)
    n_labels = len(labels)
    
    # Split each label's indices into test and train
    train_indices = []
    test_indices = []
    for i in range(n_labels):
        label_indices = np.where(y == labels[i])[0]
        label_train_indices, label_test_indices = train_test_split(label_indices, test_size=test_size, 
                                                                    random_state=random_state)
        train_indices += list(label_train_indices)
        test_indices += list(label_test_indices)
    
    # Shuffle train and test indices
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    
    # Get corresponding data and labels
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test

In [15]:
X_train, X_test, y_train, y_test = split_data_equally(shuffled_data,shuffled_labels,test_size=0.3 )

In [16]:
np.shape(X_train)

(5270, 12, 1000)

In [17]:
import torch.nn as nn

def create_signal_classifier(input_channels, num_classes, dilation_rates=[1, 2, 4]):
    # Define the convolutional layers
    conv_layers = []
    in_channels = input_channels
    for dilation_rate in dilation_rates:
        out_channels = 2 * in_channels
        conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=(1, 1),
                               padding=dilation_rate, dilation=dilation_rate)
        conv_layers.append(conv_layer)
        conv_layers.append(nn.ReLU())
        in_channels = out_channels

    # Define the fully connected layers
    fc_layers = [
        nn.Linear(in_channels * 4 * 4, 512),
        nn.ReLU(),
        nn.Linear(512, num_classes)
    ]

    # Combine the convolutional and fully connected layers into a neural network
    net = nn.Sequential(*conv_layers, nn.MaxPool2d(kernel_size=(2, 2)), nn.Flatten(), *fc_layers)

    return net


In [16]:
import tensorflow as tf
import keras
# Define the model
model = tf.keras.Sequential([
    # Convolutional layer with 32 filters, each 3x3 in size
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(12, np.shape(dataall)[2])),
    # Max pooling layer with pool size of 2
    tf.keras.layers.MaxPooling1D(pool_size=2),
    # Flatten the output of the convolutional layer
    tf.keras.layers.Flatten(),
    # Dense layer with 64 units
    tf.keras.layers.Dense(units=64, activation='relu'),
    # Output layer with 5 units (one for each label)
    tf.keras.layers.Dense(units=5, activation='softmax')
])

# # Compile the model
# optimizer = keras.optimizers.Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
model = tf.keras.Sequential([
    # Convolutional layer with 32 filters, each 3x3 in size
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(12, np.shape(dataall)[2])),
    # Max pooling layer with pool size of 2
    tf.keras.layers.MaxPooling1D(pool_size=2),
    # Additional Convolutional layer with 64 filters, each 3x3 in size
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    # Additional Max pooling layer with pool size of 2
    tf.keras.layers.MaxPooling1D(pool_size=2),
    # Flatten the output of the convolutional layer
    tf.keras.layers.Flatten(),
    # Additional Dense layer with 128 units
    tf.keras.layers.Dense(units=128, activation='relu'),
    # Additional Dropout layer with rate of 0.2
    tf.keras.layers.Dropout(0.2),
    # Dense layer with 64 units
    tf.keras.layers.Dense(units=64, activation='relu'),
    # Output layer with 5 units (one for each label)
    tf.keras.layers.Dense(units=5, activation='softmax')
])
# optimizer = keras.optimizers.Adam(lr=0.001)
# model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# # Define callbacks
# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)
# checkpoint_callback = ModelCheckpoint(filepath='model_weights.h5', save_best_only=True, save_weights_only=True)

# # Train the model with callbacks
# model.fit(X_train, y_train, epochs=20, batch_size=5, validation_split=0.2)

In [ ]:
from keras.callbacks import LearningRateScheduler

# Define a function to decay the learning rate
def lr_decay(epoch):
    initial_lr = 0.001
    decay_rate = 0.18
    decay_step = 7
    lr = initial_lr * (decay_rate ** (epoch // decay_step))
    return lr

# Create an instance of the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_decay, verbose=1)

# Create an optimizer and add the learning rate scheduler callback
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100,  batch_size=5,validation_split=0.2, callbacks=[lr_scheduler])

C:\Users\PikPik\anaconda\anaconda latest\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/100
844/844 [==============================] - 11s 9ms/step - loss: 1.4290 - accuracy: 0.3366 - val_loss: 0.8270 - val_accuracy: 0.6157 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/100
844/844 [==============================] - 4s 5ms/step - loss: 0.6870 - accuracy: 0.6914 - val_loss: 0.6250 - val_accuracy: 0.7192 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/100
844/844 [==============================] - 4s 5ms/step - loss: 0.3869 - accuracy: 0.8465 - val_loss: 0.4677 - val_accuracy: 0.7951 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/100
844/844 [==============================] - 5s 6ms/step - loss: 0.2198 - accuracy: 0.9224 - val_loss: 0.5125 - val_accuracy: 0.8046 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/100
844/844 [==============================] - 

In [ ]:
np.sum((np.argmax(model(X_test),axis=1)==y_test)*1)/2260

In [32]:
np.shape(X_test)

(2260, 12, 1000)

In [83]:
labels

array([1., 1., 1., ..., 3., 3., 3.])